<a href="https://colab.research.google.com/github/asishdash/ChatGpt-LLM/blob/main/Langchain/Model_Inputs_Outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model IO Exercise Solution

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

In [2]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


In [3]:
import os
import openai

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

#f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
#api_key = f.read()
api_key = os.environ['OPENAI_API_KEY']

In [6]:
class HistoryQuiz():

    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:

            "On what date did World War 2 end?"

        '''
        # PART ONE: SYSTEM
        system_template="You write single quiz questions about {topic}. You only return the quiz question."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        # PART TWO: HUMAN REQUEST
        human_template="{question_request}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        # PART THREE: COMPILE TO CHAT
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        # PART FOUR: INSERT VARIABLES
        request = chat_prompt.format_prompt(topic=topic,question_request="Give me a quiz question where the correct answer is a specific date.").to_messages()
        # PART FIVE: CHAT REQUEST
        chat = ChatOpenAI(openai_api_key=api_key)
        result = chat(request)


        return result.content

    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!

        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        # Datetime Parser
        output_parser = DatetimeOutputParser()

        # SYSTEM Template
        system_template = "You answer quiz questions with just a date."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)


        # HUMAN Template
        human_template = """Answer the user's question:

        {question}

        {format_instructions}"""
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # Compile ChatTemplate
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

        # Insert question and format instructions

        request = chat_prompt.format_prompt(question=question,
                                            format_instructions=output_parser.get_format_instructions()).to_messages()



        # Chat Bot
        chat = ChatOpenAI(openai_api_key=api_key)
        result = chat(request)
        # Format Request to datetime
        correct_datetime = output_parser.parse(result.content)
        return correct_datetime

    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)


        # Get the year, month, and day from the user
        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))

        # Create a datetime object
        user_datetime = datetime(year, month, day)


        return user_datetime


    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''

        # Calculate the difference between the dates
        difference = user_answer - ai_answer

        # Format the difference into a string
        formatted_difference = str(difference)

        # Return the string reporting the difference
        print("The difference between the dates is:", formatted_difference)



In [7]:
quiz_bot = HistoryQuiz()

In [8]:
question = quiz_bot.create_history_question(topic='World War 2')

In [9]:
question

'On which date did the United States enter World War II after the attack on Pearl Harbor?'

In [10]:
ai_answer = quiz_bot.get_AI_answer(question)

In [11]:
# Day After Pearl Harbor
ai_answer

datetime.datetime(1941, 12, 7, 0, 0)

In [13]:
user_answer = quiz_bot.get_user_answer(question)

On which date did the United States enter World War II after the attack on Pearl Harbor?
Enter the year: 1948
Enter the month (1-12): 12
Enter the day (1-31): 03


In [14]:
user_answer

datetime.datetime(1948, 12, 3, 0, 0)

In [15]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the dates is: 2553 days, 0:00:00
